In [ ]:
import os
import sys
import wandb
from huggingface_hub import login

In [ ]:
wandb.login()

In [ ]:
login()

In [ ]:
import torch

print(torch.cuda.is_available())

In [ ]:
#non-LORA only KD

!accelerate launch --mixed_precision="bf16"  /w/340/vishnouvina/mobilediffusion/distill_training.py \
  --pretrained_model_name_or_path="SG161222/Realistic_Vision_V4.0" \
  --dataset_name="fantasyfish/laion-art" \
  --resolution=512 --center_crop --random_flip \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --max_train_steps=15000 \
  --distill_level="sd_tiny"\
  --prepare_unet="True"\
  --mixed_precision="bf16"\
  --output_weight=0.5\
  --feature_weight=0.5\
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="mobilediffusion/models/sd-laion-art-KD"\
  --max_grad_norm=1\
  --push_to_hub \
  --report_to="wandb"\
  --resume_from_checkpoint="latest"\
  --hub_model_id="sd-laion-art-KD"

In [ ]:
!accelerate launch --mixed_precision="bf16" /w/340/vishnouvina/mobilediffusion/lora_distill_training.py \
    --pretrained_model_name_or_path="SG161222/Realistic_Vision_V4.0" \
    --dataset_name="fantasyfish/laion-art" \
    --caption_column="text"\
    --resolution=512 --center_crop --random_flip \
    --num_train_epochs=5 \
    --gradient_accumulation_steps=4 \
    --gradient_checkpointing \
    --max_train_steps=30000 \
    --validation_epochs=1 \
    --distill_level="sd_tiny"\
    --prepare_unet="True"\
    --use_peft\
    --use_8bit_adam\
    --output_weight=0.5\
    --feature_weight=0.5\
    --learning_rate=1e-05\
    --report_to="wandb"\
    --lora_r=4 \
    --lora_alpha=32 \
    --output_dir="mobilediffusion/models/sd-laion-art"\
    --max_grad_norm=1\
    --resume_from_checkpoint="latest"\
    --push_to_hub \
    --hub_model_id="sd-laion-art"

In [ ]:
!accelerate launch --mixed_precision="bf16" /w/340/vishnouvina/mobilediffusion/lora_distill_training.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1" \
    --dataset_name="fantasyfish/laion-art" \
    --caption_column="text"\
    --resolution=512 --center_crop --random_flip \
    --num_train_epochs=5 \
    --gradient_accumulation_steps=4 \
    --gradient_checkpointing \
    --max_train_steps=800 \
    --validation_epochs=1 \
    --distill_level="sd_tiny"\
    --prepare_unet="True"\
    --use_peft\
    --use_8bit_adam\
    --output_weight=0.5\
    --feature_weight=0.5\
    --learning_rate=1e-05\
    --report_to="wandb"\
    --lora_r=4 \
    --lora_alpha=32 \
    --push_to_hub \
    --hub_model_id="sd-laion-art"\
    --output_dir="sd-laion-art"\
    --max_grad_norm=1